[http://www.elmostrador.cl/]

In [2]:
from robobrowser import RoboBrowser
urls = 'http://www.elmostrador.cl/','http://www.elmostrador.cl/dia/','http://www.elmostrador.cl/dia/page/2/','http://www.elmostrador.cl/dia/page/3/'

browser = RoboBrowser(history=True, parser='html5lib')

In [3]:
articulos = []
for url in urls:
    browser.open(url)
    articulos += browser.select('h3 a') + browser.select('h2 a')

In [4]:
for ct, articulo in enumerate(articulos[:20]):
    print ct, articulo.text

0 Se cierran las apuestas: apostadores prevén rechazo al Brexit
1 Valdés propone alza de sueldo mínimo a solo $260 mil para “cautelar empleabilidad” y gatilla nuevo conflicto con la CUT
2 Miradas: ¿Es posible consumir el agua de mar? mitos y beneficios
3 Punto Rojo: La intrascendencia de las elecciones primarias
4 Chile 2016: los flancos abiertos del discurso hegemónico
5 Elecciones en España: el recurso del miedo
6 Ni la calle ni las urnas: mecanismos de participación ciudadana institucionalizados
7 Comisión del Trabajo aprobó veto presidencial a la Reforma Laboral
8 CDE pide al Ministerio Público los antecedentes sobre la investigación contra Jorge Insunza
9 Comisión de Seguridad Ciudadana de la Cámara aprobó idea de legislar la “ley antiencapuchados”
10 [VIDEO] La funa al ex jefe de Gabinete de Cristina Fernández en un vuelo a Londres
11 PPD no le presta ropa a Insunza tras investigación por cohecho que abrió la Fiscalía en su contra
12 Hernán Larraín Matte: “Es voluntad del Consejo

In [5]:
articulo=articulos[9]
print articulo

<a href="http://www.elmostrador.cl/noticias/pais/2016/06/21/comision-de-seguridad-ciudadana-de-la-camara-aprobo-idea-de-legislar-la-ley-antiencapuchados/" title="Ir a enlace">Comisión de Seguridad Ciudadana de la Cámara aprobó idea de legislar la “ley antiencapuchados”</a>


In [6]:
browser.follow_link(articulo)
browser.select('.datos-noticias h2')

[<h2>Comisi\xf3n de Seguridad Ciudadana de la C\xe1mara aprob\xf3 idea de legislar la "ley antiencapuchados"</h2>]

In [7]:
texto = browser.select('.cuerpo-noticia')[0].text
titulo = browser.select('.datos-noticias h2')[0].text
print titulo, texto

Comisión de Seguridad Ciudadana de la Cámara aprobó idea de legislar la "ley antiencapuchados" 
													
																		                                    Este martes, por 7 votos a favor y 1 en contra, la Comisión de Seguridad Ciudadana aprobó en general la idea de legislar el proyecto de ley conocido como "ley antiencapuchados", el cual modifica el artículo 494 del Código Penal, estableciendo la posibilidad de detener a quien "con ocasión de alteraciones al orden público, oculte su rostro para evitar su identificación", sumando además una multa de entre 1 a 4 UTM.
Tras la sesión, el presidente de la Comisión, Daniel Farcas (PPD), comentó que "junto a todos los parlamentarios con los que presentamos este proyecto, tenemos la convicción de que a partir de hoy los encapuchados tienen los días contados en el país, pues si bien el derecho de reunión o manifestación es algo sagrado y muy importante, en ningún caso puede verse empañado por estos vándalos que concurren a las marchas

In [8]:
import indicoio
indicoio.config.api_key = 'anda a indico.io y obten tu key y ponlo acá'
people = indicoio.people(texto)
for p in people[:15]:
    print p

{u'text': u'este', u'confidence': 0.5978337526, u'position': [2996, 3000]}
{u'text': u'este', u'confidence': 0.4770954251, u'position': [624, 628]}
{u'text': u'este', u'confidence': 0.3748088479, u'position': [2124, 2128]}
{u'text': u'Este', u'confidence': 0.34479981660000003, u'position': [69, 73]}
{u'text': u'Daniel Farcas', u'confidence': 0.3438846767, u'position': [533, 546]}
{u'text': u'Mario Fern\xe1ndez', u'confidence': 0.3109764159, u'position': [2877, 2892]}
{u'text': u'este', u'confidence': 0.2645292282, u'position': [1620, 1624]}
{u'text': u'Seg\xfan Soto', u'confidence': 0.2529143095, u'position': [2476, 2486]}
{u'text': u'este', u'confidence': 0.2210849226, u'position': [1893, 1897]}
{u'text': u'Nicol\xe1s Eyzaguirre', u'confidence': 0.2132452279, u'position': [2911, 2929]}
{u'text': u'cuya', u'confidence': 0.18606027960000002, u'position': [2089, 2093]}
{u'text': u'Nogueira', u'confidence': 0.1770956367, u'position': [2704, 2712]}
{u'text': u'este', u'confidence': 0.15841

/Users/rodrigo/Envs/datavis/lib/python2.7/site-packages/indicoio/utils/api.py:49: UserWarning: people API version 1 is going to be deprecated by September 17, 2016. Please upgrade to version 2
  warnings.warn(warning)


In [9]:
from string import uppercase

def is_person(txt):
    partes = txt.split(' ')
    if len(partes)<2:
        return False
    for parte in partes:
        try:
            if parte[0] not in uppercase:
                return False
        except:
            return False
    return True

print is_person('hoy ayer')

False


In [10]:
people = indicoio.people(texto)
for p in people:
    if is_person(p['text']):
        print p['text']
    

Daniel Farcas
Mario Fernández
Según Soto
Nicolás Eyzaguirre
Leonardo Soto
Marcela Sabat
Claudia Nogueira


![grafo](graph.png)

In [11]:
import networkx as nx
G=nx.Graph()

print 'empezamos'
for articulo in articulos:
    if '2016' not in articulo.attrs['href']:
        continue
    browser.follow_link(articulo)
    texto = browser.select('.cuerpo-noticia')
    if texto:
        texto = texto[0].text
    else:
        continue        
    try:
        people = indicoio.people(texto, threshold=0.22)
    except:
        continue
    palabras = []
    for person in people:
        nombre = person['text']
        if is_person(nombre):
            palabras.append(nombre)            
    for palabra1 in set(palabras):
        G.add_node(palabra1)
        for palabra2 in set(palabras):
            if palabra1 != palabra2:
                G.add_edge(palabra1, palabra2)
                print palabra1, '-', palabra2
print "terminamos"

empezamos
Rodrigo Valdés - Ximena Rincón
Ximena Rincón - Rodrigo Valdés
Bárbara Figueroa - Camila Vallejo
Bárbara Figueroa - Iván Fuentes
Bárbara Figueroa - Giorgio Jackson
Bárbara Figueroa - Karol Cariola
Camila Vallejo - Bárbara Figueroa
Camila Vallejo - Iván Fuentes
Camila Vallejo - Giorgio Jackson
Camila Vallejo - Karol Cariola
Iván Fuentes - Bárbara Figueroa
Iván Fuentes - Camila Vallejo
Iván Fuentes - Giorgio Jackson
Iván Fuentes - Karol Cariola
Giorgio Jackson - Bárbara Figueroa
Giorgio Jackson - Camila Vallejo
Giorgio Jackson - Iván Fuentes
Giorgio Jackson - Karol Cariola
Karol Cariola - Bárbara Figueroa
Karol Cariola - Camila Vallejo
Karol Cariola - Iván Fuentes
Karol Cariola - Giorgio Jackson
Mariano Rajoy - Felipe González
Mariano Rajoy - Pablo Iglesias
Mariano Rajoy - Susana Díaz
Mariano Rajoy - Pedro Sánchez
Felipe González - Mariano Rajoy
Felipe González - Pablo Iglesias
Felipe González - Susana Díaz
Felipe González - Pedro Sánchez
Pablo Iglesias - Mariano Rajoy
Pablo Igl

In [12]:
for i in G.nodes()[:20]:
    print i

Martin Heidegger
Luis Mariano Rendón
Roberto Lecaros
Julio Dittborn Cordua
Ximena Rincón
Mauricio Isla
Cristóbal Saavedra
Daniela Guerrero
Jean Phillipe Creton
Charles Aránguiz
Edson Puch
Rubén Valenzuela
Alexis Sánchez
Ricardo Lagos
Cristian Zapata
Carlos Bacca
Pablo Longueira
Alejandro Guillier
Patricio Vallespín
Eva Ayllón


In [13]:
for i in G.edges()[:20]:
    print i

(u'Martin Heidegger', u'Diego Arra\xf1o')
(u'Martin Heidegger', u'Sigmund Freud')
(u'Martin Heidegger', u'Francisco Javier Cuadra')
(u'Luis Mariano Rend\xf3n', u'Michelle Bachelet')
(u'Luis Mariano Rend\xf3n', u'Ricardo Lagos')
(u'Roberto Lecaros', u'Daniel Lencina')
(u'Roberto Lecaros', u'Alfredo Espinoza')
(u'Roberto Lecaros', u'Felipe Mella')
(u'Roberto Lecaros', u'Pedro Greene')
(u'Roberto Lecaros', u'Christian G\xe1lvez')
(u'Roberto Lecaros', u'Cristi\xe1n Cuturrufo')
(u'Julio Dittborn Cordua', u'Jaime Orpis')
(u'Julio Dittborn Cordua', u'Hern\xe1n Larra\xedn')
(u'Julio Dittborn Cordua', u'Jorge Insunza')
(u'Ximena Rinc\xf3n', u'Rodrigo Vald\xe9s')
(u'Ximena Rinc\xf3n', u'Patricio Vallesp\xedn')
(u'Mauricio Isla', u'Carlos S\xe1nchez')
(u'Mauricio Isla', u'Edwin Cardona')
(u'Mauricio Isla', u'Juan Guillermo Cuadrado')
(u'Mauricio Isla', u'Joel Aguilar')


In [14]:
import json
from networkx.readwrite import json_graph
 
for n in G:
    G.node[n]['name'] = n
 
d = json_graph.node_link_data(G)
json.dump(d, open('force/force.json','w'), sort_keys=True, indent=4, separators=(',', ': '))

[json 1](force/force.json)

[grafo 1](force/force.html)

In [15]:
to_remove = []
G2 = G.copy()
for i in nx.connected_component_subgraphs(G2):
    c = nx.average_clustering(i)
    if c==1 or c==0:
        to_remove.append(i)
        
for i in to_remove:        
    G2.remove_nodes_from(i)

for n in G2:
    G2.node[n]['name'] = n
 
d = json_graph.node_link_data(G2)
json.dump(d, open('force2/force.json','w'), sort_keys=True, indent=4, separators=(',', ': '))

[json 2](force2/force.json)

[grafo 2](force2/force.html)